In [9]:
import os
import sys
import gzip
import code
import urllib
import numpy as np
from tqdm import tqdm
from PIL import Image

import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split

In [10]:
%reload_ext autoreload
%autoreload 2

sys.path.append('./')
from helpers.helpers import *
from helpers.mask_to_submission import *

In [11]:
PREDICTIONS_SAVE_DIR = 'predictions/'
MODELS_SAVE_DIR = 'model_save/'

TRAINING_SAMPLES = 100 #max 100

TRAINING_SIZE = 80 # Size of the training set in percentage, integer between 0 and 100, the remaining part is for testing
VALIDATION_SIZE = 0.20  # Size of the validation set, float between 0 and 1
SEED = 66478  # Set to None for random seed.
NUM_EPOCHS = 15

NUM_CHANNELS = 3  # RGB images
PIXEL_DEPTH = 255
NUM_LABELS = 2
BATCH_SIZE = 16  # 64
RECORDING_STEP = 0

# Set image patch size in pixels
# IMG_PATCH_SIZE should be a multiple of 4
# image size should be an integer multiple of this number!
IMG_PATCH_SIZE = 16
IMAGE_SIZE = 400
if IMAGE_SIZE % IMG_PATCH_SIZE != 0 :
    print('Warning : the image size is not a multiple of the patch size')

In [12]:
data_dir = 'data/training/'
train_data_filename = data_dir + 'images/'
train_labels_filename = data_dir + 'groundtruth/' 

def extract_data(filename, num_images):
    imgs = []
    for i in tqdm(range(1, num_images+1)):
        imageid = "satImage_%.3d" % i
        image_filename = filename + imageid + ".png"
        if os.path.isfile(image_filename):
            img = mpimg.imread(image_filename)
            imgs.append(img)
        else:
            print('File ' + image_filename + ' does not exist')
            
    print('Loaded {} training images'.format(len(imgs)))
    return np.asarray(imgs)

images = extract_data(train_data_filename, TRAINING_SAMPLES)
groundtruths = extract_data(train_labels_filename, TRAINING_SAMPLES)

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

Loaded 100 training images


100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 465.12it/s]

Loaded 100 training images


In [13]:
print(images[0].shape)
images_mean = images#.mean(axis=3)
print(images_mean[0].shape)
print(groundtruths[0].shape)

(400, 400, 3)
(400, 400, 3)
(400, 400)


In [14]:
import gc

def train(model, criterion, dataset_train, y_train, dataset_test, y_test, optimizer, num_epochs, device):
  """
  @param model: torch.nn.Module
  @param criterion: torch.nn.modules.loss._Loss
  @param dataset_train: torch.utils.data.DataLoader
  @param dataset_test: torch.utils.data.DataLoader
  @param optimizer: torch.optim.Optimizer
  @param num_epochs: int
  """
  print("Starting training")
  for epoch in range(num_epochs):
    # Train an epoch
    print('epoch number ',epoch)
    model.train()
    batch_x, batch_y = dataset_train, y_test

    # Evaluate the network (forward pass)
    prediction = model(batch_x)
    loss = criterion(prediction, batch_y)

    # Compute the gradient
    optimizer.zero_grad()
    loss.backward()

    # Update the parameters of the model with a gradient step
    optimizer.step()

    # Test the quality on the test set
    model.eval()
    accuracies_test = []

    batch_x, batch_y = dataset_test, y_test

    # Evaluate the network (forward pass)
    prediction = model(batch_x)
    accuracies_test.append(accuracy(prediction, batch_y))
    gc.collect()
    torch.cuda.empty_cache()

    print("Epoch {} | Test accuracy: {:.5f}".format(epoch, sum(accuracies_test).item()/len(accuracies_test)))

In [16]:
from scripts.unet_model import *
from scripts.train import train_net

unet = UNet(3, 2)
if not torch.cuda.is_available():
    print('Training will be ')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
unet
print('Ok')

ModuleNotFoundError: No module named 'torch'

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(unet.parameters(), lr=1e-3)
def reshape(array):
    return np.moveaxis(array, 3, 1)
print(X_train.shape)
print(y_train.shape)
train(unet, criterion, torch.tensor(reshape(X_train)), torch.tensor(y_train), torch.tensor(reshape(X_test)), torch.tensor(y_test), optimizer, NUM_EPOCHS, device)